<a href="https://colab.research.google.com/github/TurkuNLP/Text_Mining_Course/blob/master/text_sim_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text similarity / neural models

* TF-IDF does not take into account semantic similarity / paraphrasing
* Paraphrase: same meaning, different wording
* Low lexical overlap (different wording) means erroneously low TF-IDF similarity
* In the ideal case, one would capture meaning regardless of wording
* This is ongoing research, no silver bullet solutions, but steady progress in this direction can be observed!

# BERT -based similarity

* The BERT model can be seen as a device to turn input text into dense vector representation
* We will see it does not capture paraphrasing all that well, but we can use it as a suitable model to learn how to manipulate embeddings produced by neural models and gain intuition into the model's out-of-the-box capabilities


In [1]:
!wget http://dl.turkunlp.org/textual-data-analysis-course-data/hs_yle_spring_2020.json.gz

--2021-03-09 20:15:35--  http://dl.turkunlp.org/textual-data-analysis-course-data/hs_yle_spring_2020.json.gz
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20590829 (20M) [application/octet-stream]
Saving to: ‘hs_yle_spring_2020.json.gz’

hs_yle_spring_2020. 100%[===================>]  19.64M  6.99MB/s    in 2.8s    

2021-03-09 20:15:38 (6.99 MB/s) - ‘hs_yle_spring_2020.json.gz’ saved [20590829/20590829]



In [2]:
import json
import gzip
from pprint import pprint  #pprint is prettyprint

with gzip.open("hs_yle_spring_2020.json.gz") as f:
    news_data=json.load(f)


In [3]:
yle=news_data["2020"]["01"]["yle-text"]
hs=news_data["2020"]["01"]["hs-text"]

#Let us split the data into sentences...
!pip3 install ufal.udpipe


     |████████████████████████████████| 307kB 17.7MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626553 sha256=ed98775a1b7dac63ddba3dcaa6f82ed5f6a2d3636ba117cccb2259cedaf68a08
  Stored in directory: /root/.cache/pip/wheels/0c/9d/db/6d3404c33da5b7adb6c6972853efb6a27649d3ba15f7e9bebb
Successfully built ufal.udpipe


In [4]:
!wget -nc -O fi_model.udpipe https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/finnish-tdt-ud-2.5-191206.udpipe?sequence=25&isAllowed=y


--2021-03-09 20:18:32--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/finnish-tdt-ud-2.5-191206.udpipe?sequence=25
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21613253 (21M) [application/octet-stream]
Saving to: ‘fi_model.udpipe’

fi_model.udpipe     100%[===================>]  20.61M  15.4MB/s    in 1.3s    

2021-03-09 20:18:34 (15.4 MB/s) - ‘fi_model.udpipe’ saved [21613253/21613253]



In [5]:
import ufal.udpipe as udpipe

model = udpipe.Model.load("fi_model.udpipe")
pipeline = udpipe.Pipeline(model,"tokenize","none","none","horizontal")


In [6]:
print(pipeline.process(yle[0]["text"]))

Helsingin Kansalaistorin juhlat sujuivat mallikkaasti – poliisi torui muualla kaupungissa nuoria rakettien ampumisesta ihmisiä päin Helsingin Kansalaistorille oli kerääntynyt juhlimaan vuoden vaihtumista arviolta 85 000 ihmistä .
Helsingin poliisi kertoo saaneensa kymmeniä ilmoituksia ilotulitteiden väärinkäytöksistä ympäri kaupunkia .
Poliisin johtokeskuksesta kerrottiin yöllä , että poliisi oli saanut iltakymmeneen mennessä kymmeniä ilmoituksia väärinkäytöksistä .
Poliisin mukaan nuorisoporukat ovat ampuneet ilotulitteita ihmisiä , autoja ja rakennuksia päin .
Ilotulitteita ammuttiin myös sellaisilla alueilla , missä niiden ampuminen on kiellettyä .
Kansalaistorilla noin 85 000 ihmistä Helsingin kansalaistorilla järjestettiin musiikkia ja ilotulituksen sisältävä uudenvuoden juhla .
Järjestäjän arvion mukaan Kansalaistorille oli kerääntynyt juhlimaan vuoden vaihtumista arviolta 85 000 ihmistä .
Juhlinta sujui mallikkaasti , eikä poliisin tietoon tullut juhlapaikalta ilmoituksia vakava

In [7]:
import tqdm
for d in tqdm.tqdm(yle[:len(yle)//4]):
    d["segmented"]=pipeline.process(d["text"]).strip().split("\n")[:4]
for d in tqdm.tqdm(hs[:len(hs)//4]):
    d["segmented"]=pipeline.process(d["text"]).strip().split("\n")[:4]

100%|██████████| 1770/1770 [00:31<00:00, 56.52it/s]


In [9]:
print(yle[0]["segmented"])

['Helsingin Kansalaistorin juhlat sujuivat mallikkaasti – poliisi torui muualla kaupungissa nuoria rakettien ampumisesta ihmisiä päin Helsingin Kansalaistorille oli kerääntynyt juhlimaan vuoden vaihtumista arviolta 85\xa0000 ihmistä .', 'Helsingin poliisi kertoo saaneensa kymmeniä ilmoituksia ilotulitteiden väärinkäytöksistä ympäri kaupunkia .', 'Poliisin johtokeskuksesta kerrottiin yöllä , että poliisi oli saanut iltakymmeneen mennessä kymmeniä ilmoituksia väärinkäytöksistä .', 'Poliisin mukaan nuorisoporukat ovat ampuneet ilotulitteita ihmisiä , autoja ja rakennuksia päin .']


In [17]:
all_sentences=[]
for d in yle:
    all_sentences.extend(d.get("segmented",[]))
for d in hs:
    all_sentences.extend(d.get("segmented",[]))
unique_sentences=list(set(all_sentences))
unique_sentences.sort()
print("All unique sentences",len(unique_sentences))
for s in unique_sentences[:10]:
    print(s)

All unique sentences 5331

" Ei työssäkäyvän ihmisen talous siihen kaadu "
" Esimiehet pyörisivät yksin töissä ilman työntekijöitä "
" Hankitaan vain kirjoja , jotka ovat saaneet megakohun aikaan "
" Huolestunut vihje voi tulla pankista tai kaupan kassalta "
" Huono yhteistyö ei ole laitonta , se on vain hankalaa "
" Iskujen tarkoituksena ei ollut tappaa USA:n sotilaita , mutta operaatio amerikkalaisjoukkojen pois ajamiseksi jatkuu "
" Juoksi keittiöön ja nappasi leipäveitsen käteen " – vartijat joutuvat yhä useammin turvaamaan kotihoidon työntekijöiden kotikäyntejä
" Lasten ja vanhempien mielikuvitus on laiskistunut " – 8 vinkkiä lumettomiin lomapäiviin
" Menestystarina on auennut "


In [11]:
!pip3 install transformers
import transformers

     |████████████████████████████████| 1.9MB 19.6MB/s 
     |████████████████████████████████| 3.2MB 47.6MB/s 
     |████████████████████████████████| 890kB 52.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=b023853e1efbc1e4cc27fbfa183da4a202dfa788eb17ee60a4aee05ad4268eef
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [12]:
bert_tokenizer=transformers.BertTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

In [15]:
sents_tokenized=bert_tokenizer(unique_sentences,padding=True,truncation=True,return_tensors="pt")

In [18]:
print(list(sents_tokenized.keys()))
print(sents_tokenized["input_ids"].shape)

['input_ids', 'token_type_ids', 'attention_mask']
torch.Size([5331, 125])


In [19]:
import torch
ds=torch.utils.data.TensorDataset(sents_tokenized["input_ids"],sents_tokenized["token_type_ids"],sents_tokenized["attention_mask"])
batched_ds=torch.utils.data.DataLoader(ds,batch_size=8)

In [20]:
for item in batched_ds:
    print(item)
    break

[tensor([[  102,   103,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
      

In [21]:
model=transformers.BertModel.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

In [24]:
model=model.cuda() #move to GPU (you need to run the notebook on a GPU-accelerated instance, or else this crashes!)
model.eval()
all_vectors=[]
with torch.no_grad():
    for input_ids,token_type_ids,att_mask in tqdm.tqdm(batched_ds):
        input_ids=input_ids.cuda()
        token_type_ids=token_type_ids.cuda()
        att_mask=att_mask.cuda()
        model_out=model(input_ids=input_ids,token_type_ids=token_type_ids,attention_mask=att_mask)
        all_vectors.append(model_out.pooler_output.cpu())


100%|██████████| 667/667 [00:38<00:00, 17.44it/s]


In [30]:
embedded=torch.vstack(all_vectors).numpy()
print(embedded.shape)
#...and now we are in a familiar territory

(5331, 768)


In [31]:
#2) Compare
import sklearn.metrics.pairwise as pairwise

sent_sims=pairwise.cosine_similarity(embedded) #can it be made any easier than this?!
print(sent_sims.shape) #we now have all YLE-vs-HS cosine similarities :)


(5331, 5331)


In [34]:
#3) Pick most similar
# 
# This is easy, in the end, but needs some amount of numpy magic ;)

import numpy as np
sorted_indices=np.argsort(-sent_sims)[:,1:2] #we cannot take the first, because that would be the sentence itself :D
# argsort (argument sort, gives indices rather than sorted values)
# sort is always ascending but we want descending, the solution is to sort -yle_hs_sims
# [:,:1] means "take all rows and the first column" but do keep as a 2-dim array  [:,0] would produce a 1-dim array
print("Sorted_indices shape",sorted_indices.shape) #as many rows as there are YLE articles, and the index of the most similar HS article
print("First ten sorted indices",sorted_indices[:10])

#But now we want to see the YLE articles that have the highest correspondence to any HS article
#for that we need to sort again. For that, we also need the scores!
scores=np.take_along_axis(sent_sims,sorted_indices,-1)  #pick values from yle_hs_sims using the sorted_indices, on the last axis (does your head spin?)
print("scores.shape",scores.shape)
scores_sorted_indices=np.argsort(-scores.flatten()) #We need to flatten before sort or else the 2nd dimension (which has only one element) will get sorted
#this is now indices to YLE texts sorted in descending order by their similarity to any HS article



Sorted_indices shape (5331, 1)
First ten sorted indices [[3013]
 [   2]
 [   1]
 [   5]
 [   7]
 [   1]
 [5329]
 [5111]
 [  32]
 [  13]]
scores.shape (5331, 1)


In [37]:
#4) Inspect!

#Can we convince ourselves this works?
for i in scores_sorted_indices[:100]: #first 100 sentences
    #Which is the corresponding one?
    j=sorted_indices[i][0] #so which is the HS index? look it up in sorted_indices, and since that is a 2-dim array, pick the first column (numpy arrays can be head-spinning experience)
    print("------------------------------------------")
    print("yle_i",yle_i,"hs_i",hs_i) #now we know which row (YLE) and column (HS) we are referring to
    sim=yle_hs_sims[yle_i,hs_i] #this is the similarity
    print("Sim",sim)
    print("*********** YLE")
    print(yle_texts[yle_i][:500]) #this is the YLE article, first 500 chars
    print("*********** HS")
    print(hs_texts[hs_i][:500]) #...and this is the HS article, first 500 chars
    print("------------------------------------------")
    print()

------------------------------------------
yle_i 1290 hs_i 4710
Sim 0.975945391128141
*********** YLE
Helsingin Laakson sairaalan lääkärit pelkäävät potilasturvallisuuden vaarantuvan lääkäripulan vuoksi – myös osastojen sulkeminen vaihtoehtona
"Nyt olemme kriisitilanteessa", lääkärit sanovat. Lääkäreistä on vajetta ympäri Suomen.
Helsingin Laakson sairaalan lääkärit ovat erittäin huolissaan sairaalan pitkään jatkuneesta ja yhä pahenevasta lääkäripulasta.
Lääkärien mukaan potilasturvallisuus on vaarantunut toistuvasti.
Laakson sairaalan lääkärit ovat lähestyneet Helsingin kaupunkia ja myös työsuo
*********** HS
Helsingin Laakson sairaalan lääkärit ovat erittäin huolissaan sairaalan pitkään jatkuneesta ja yhä pahenevasta lääkäripulasta.
Lääkärien mukaan potilasturvallisuus on vaarantunut toistuvasti.
Laakson sairaalan lääkärit ovat lähestyneet Helsingin kaupunkia ja myös työsuojelua kirjelmillä, joissa kerrotaan lääkäripulan vaikutuksista.
”Tämän seurauksena me apulaisylilääkärit ja osas

# That worked like charm!